# Movie Review Examples

Let's walk through a simple example of solving tasks using spark tables and LLMs. Imagine we have a table of movie reviews:

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # for table pretty printing

data = [
    {"name": "A. Smith", "age": 20, "review": "The movie was great!"},
    {"name": "B. Jones", "age": 35, "review": "The movie did not live up to the hype."},
    {"name": "C. Addams", "age": 40, "review": "Why is this movie so bad when it was supposed to be good?"},
]

df = spark.createDataFrame(data)
df

age,name,review
20,A. Smith,The movie was great!
35,B. Jones,The movie did not...
40,C. Addams,Why is this movie...


We want to use an LLM to decide whether each review is positive or negative. Let's add a new column, `prompt` which takes the `review` column and prepends "Is this review positive or negative":

In [4]:
import pyspark.sql.functions as F

df = df.withColumn(
    "prompt",
    F.concat(
        F.lit("Is this review positive or negative?\n"),
        F.col("review"),
    ),
)
df

age,name,review,prompt
20,A. Smith,The movie was great!,Is this review po...
35,B. Jones,The movie did not...,Is this review po...
40,C. Addams,Why is this movie...,Is this review po...


Setup the openai client with your API key:

In [6]:
from openai import OpenAI
client = OpenAI()

Now let's import the `spark_batch_ai` library and process the table:

In [8]:
from pyspark_batch_ai import process_dataframe

df_with_result = process_dataframe(df, client, model="gpt-3.5-turbo-0125")
df_with_result

2025-01-08 15:58:46.077 | INFO     | pyspark_batch_ai.core:process_dataframe:120 - Detected output format: plain
2025-01-08 15:58:46.267 | INFO     | pyspark_batch_ai.core:_submit_and_process:366 - Total number of jobs to run: 1
2025-01-08 15:58:49.150 | INFO     | pyspark_batch_ai.core:_submit_and_process:374 - Currently running: 1, Jobs left in queue: 0
2025-01-08 15:59:50.026 | INFO     | pyspark_batch_ai.core:monitor_batches:319 - Batch ID: batch_677ea0b8b35c8190b75ca17e15d01906, Status changed from validating to completed
2025-01-08 15:59:50.026 | INFO     | pyspark_batch_ai.core:monitor_batches:330 - Batch ID batch_677ea0b8b35c8190b75ca17e15d01906 completed. Output file ID: file-4Etiqu4ikmzcb9SHx94Wfq
2025-01-08 15:59:50.026 | INFO     | pyspark_batch_ai.core:monitor_batches:335 - Jobs left to process: 0
2025-01-08 15:59:50.027 | INFO     | pyspark_batch_ai.core:monitor_batches:351 - All batch jobs have completed.
2025-01-08 15:59:50.027 | INFO     | pyspark_batch_ai.core:_submit

age,name,review,prompt,response
20,A. Smith,The movie was great!,Is this review po...,positive
35,B. Jones,The movie did not...,Is this review po...,negative
40,C. Addams,Why is this movie...,Is this review po...,Negative\n\nThe r...
